In [1]:
from Datasets import BaselineDataset

In [2]:
# Ensure data directory is populated with https://drive.google.com/drive/folders/1pjHyFlThbpXcvldabLOZZwf4dHaM-p54?usp=share_link

ticker = "AAPL"
data_dir = "../data"
baseline_dataset = BaselineDataset(ticker, data_dir)

In [3]:
len(baseline_dataset) # should be # of dates minus 7

2427

In [4]:
baseline_dataset[0]

(tensor([22.4500, 22.7700, 23.0300, 23.1200, 23.0600, 23.4200, 23.5600]),
 tensor(23.4700))